In [1]:
using Oscar

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  Version 1.1.1


# Matroid Basics

The following matroid is Example 10.6 in [Essentials of tropical combinatorics](https://bookstore.ams.org/gsm-219/).

In [2]:
M = matroid_from_bases([[1,2,3], [1,2,4], [1,2,5], [1,3,4], [1,4,5], [2,3,5], [2,4,5], [3,4,5]], 5)

Matroid of rank 3 on 5 elements

A matroid is uniquely determined by its circuits ...

In [3]:
circuits(M)

3-element Vector{Vector{Int64}}:
 [1, 3, 5]
 [2, 3, 4]
 [1, 2, 4, 5]

... or its flats.

In [4]:
flats(M)

13-element Vector{Vector{Int64}}:
 []
 [1]
 [2]
 [3]
 [4]
 [5]
 [1, 2]
 [1, 4]
 [1, 3, 5]
 [2, 3, 4]
 [2, 5]
 [4, 5]
 [1, 2, 3, 4, 5]

A proper flat is <i>cyclic</i> if it is the disjoint union of circuits.  Again, the cyclic flats are enough to define the matroid uniquely.  Often they form a particularly compact encoding.

In [5]:
cyclic_flats(M)

4-element Vector{Vector{Int64}}:
 []
 [1, 3, 5]
 [2, 3, 4]
 [1, 2, 3, 4, 5]

The independent sets form an abstract simplicial complex.

In [6]:
K = simplicial_complex(independent_sets(M))

Abstract simplicial complex of dimension 2 on 5 vertices

In [7]:
f_vector(K)

3-element Vector{Int64}:
  5
 10
  8

In [8]:
[ homology(K, i) for i in 0:2 ]

3-element Vector{FinGenAbGroup}:
 Z
 Z/1
 Z^2

In [9]:
P = matroid_base_polytope(M);
vertices(P)

8-element SubObjectIterator{PointVector{QQFieldElem}}:
 [1, 1, 1, 0, 0]
 [1, 1, 0, 1, 0]
 [1, 1, 0, 0, 1]
 [1, 0, 1, 1, 0]
 [1, 0, 0, 1, 1]
 [0, 1, 1, 0, 1]
 [0, 1, 0, 1, 1]
 [0, 0, 1, 1, 1]

In [10]:
dim(P) == length(matroid_groundset(M)) - length(connected_components(M))

true

# Chow Rings

The discussion below follows the chapter "Matroids" by Corey, Kühne & Schröter in [The Computer Algebra System OSCAR, Springer (2024)](https://link.springer.com/book/9783031621260).

## Characteristic polynomials

In [11]:
M = cycle_matroid(complete_graph(4))

Matroid of rank 3 on 6 elements

In [12]:
tutte_polynomial(M)

x^3 + 3*x^2 + 4*x*y + 2*x + y^3 + 3*y^2 + 2*y

In [13]:
char_poly = characteristic_polynomial(M)

q^3 - 6*q^2 + 11*q - 6

In [14]:
factor(char_poly)

1 * (q - 2) * (q - 1) * (q - 3)

The absolute values of the coefficients ...

In [15]:
w = [ abs(coeff(char_poly, i-1)) for i in 1:length(char_poly) ]

4-element Vector{ZZRingElem}:
 6
 11
 6
 1

... satisfy log-concavity.

In [16]:
w[1]*w[3] <= w[2]^2 && w[2]*w[4] <= w[3]^2

true

In [17]:
flats(M)

15-element Vector{Vector{Edge}}:
 []
 [Edge(2, 1)]
 [Edge(3, 1)]
 [Edge(3, 2)]
 [Edge(4, 1)]
 [Edge(4, 2)]
 [Edge(4, 3)]
 [Edge(2, 1), Edge(3, 1), Edge(3, 2)]
 [Edge(2, 1), Edge(4, 1), Edge(4, 2)]
 [Edge(2, 1), Edge(4, 3)]
 [Edge(3, 1), Edge(4, 2)]
 [Edge(3, 1), Edge(4, 1), Edge(4, 3)]
 [Edge(3, 2), Edge(4, 1)]
 [Edge(3, 2), Edge(4, 2), Edge(4, 3)]
 [Edge(2, 1), Edge(3, 1), Edge(3, 2), Edge(4, 1), Edge(4, 2), Edge(4, 3)]

## Definition of the Chow ring

In [18]:
A = chow_ring(M)

Quotient
  of multivariate polynomial ring in 13 variables x_{Edge(2, 1)}, x_{Edge(3, 1)}, x_{Edge(3, 2)}, x_{Edge(4, 1)}, ..., x_{Edge(3, 2),Edge(4, 2),Edge(4, 3)}
    over rational field
  by ideal with 65 generators

In [19]:
GR, _ = graded_polynomial_ring(QQ, symbols(base_ring(A)));
AA = chow_ring(M, ring=GR);

In [20]:
vol_map = volume_map(M, AA)

#3982 (generic function with 1 method)

In [21]:
e = matroid_groundset(M)[1];

In [22]:
proper_flats = flats(M)[2:length(flats(M))-1];

In [23]:
a = sum([AA[i] for i in 1:length(proper_flats) if e in proper_flats[i]])

x_{Edge(2, 1)} + x_{Edge(2, 1),Edge(3, 1),Edge(3, 2)} + x_{Edge(2, 1),Edge(4, 1),Edge(4, 2)} + x_{Edge(2, 1),Edge(4, 3)}

In [24]:
b = sum([AA[i] for i in 1:length(proper_flats) if !(e in proper_flats[i])])

x_{Edge(3, 1)} + x_{Edge(3, 2)} + x_{Edge(4, 1)} + x_{Edge(4, 2)} + x_{Edge(4, 3)} + x_{Edge(3, 1),Edge(4, 2)} + x_{Edge(3, 1),Edge(4, 1),Edge(4, 3)} + x_{Edge(3, 2),Edge(4, 1)} + x_{Edge(3, 2),Edge(4, 2),Edge(4, 3)}

In [25]:
a*b

-5*x_{Edge(3, 2),Edge(4, 2),Edge(4, 3)}^2

## Poincaré duality

In [26]:
k = 1

R = base_ring(AA);
g = grading_group(R)[1];
PD1, mapPD1 = homogeneous_component(AA, k*g);
basis_PD1 = [mapPD1(x) for x in gens(PD1)];
PD2, mapPD2 = homogeneous_component(AA,(rank(M)-k-1)*g);
basis_PD2 = [mapPD2(x) for x in gens(PD2)];

In [27]:
Mat1 = matrix(QQ,[[vol_map(b1*b2) for b1 in basis_PD1] for b2 in basis_PD2])

[-1    0    0    0    0    0    0    1]
[ 0   -1    0    0    0    0    0    0]
[ 0    0   -1    0    0    0    0    1]
[ 0    0    0   -1    0    0    0    0]
[ 0    0    0    0   -1    0    0    1]
[ 0    0    0    0    0   -1    0    0]
[ 0    0    0    0    0    0   -1    0]
[ 1    0    1    0    1    0    0   -2]

In [28]:
rank(Mat1)

8

## Hard Lefschetz property

In [29]:
Mat2 = matrix(QQ,[[vol_map(b1*b^(rank(M)-2k-1)*b2) for b1 in basis_PD1] for b2 in basis_PD1]);

In [30]:
Mat1 == Mat2

true

## Hodge-Riemann relations

In [31]:
RR, _ = graded_polynomial_ring(QQ, "y_#" => 1:length(basis_PD1));
map = hom(RR, AA, basis_PD1)

Ring homomorphism
  from graded multivariate polynomial ring in 8 variables over QQ
  to quotient of multivariate polynomial ring by ideal with 65 generators
defined by
  y_1 -> x_{Edge(3, 2),Edge(4, 2),Edge(4, 3)}
  y_2 -> x_{Edge(3, 2),Edge(4, 1)}
  y_3 -> x_{Edge(3, 1),Edge(4, 1),Edge(4, 3)}
  y_4 -> x_{Edge(3, 1),Edge(4, 2)}
  y_5 -> x_{Edge(2, 1),Edge(4, 3)}
  y_6 -> x_{Edge(2, 1),Edge(4, 1),Edge(4, 2)}
  y_7 -> x_{Edge(2, 1),Edge(3, 1),Edge(3, 2)}
  y_8 -> x_{Edge(4, 3)}

In [32]:
K = kernel(hom(RR, AA, [b1^(rank(M)-2k)*b1 for b1 in basis_PD1]));

In [33]:
basis_HR = [map(h) for h in gens(K) if degree(h).coeff==k*g.coeff]

7-element Vector{MPolyQuoRingElem{MPolyDecRingElem{QQFieldElem, QQMPolyRingElem}}}:
 -x_{Edge(4, 3)} + 2*x_{Edge(2, 1),Edge(3, 1),Edge(3, 2)}
 -x_{Edge(4, 3)} + 2*x_{Edge(2, 1),Edge(4, 1),Edge(4, 2)}
 -x_{Edge(4, 3)} + 2*x_{Edge(2, 1),Edge(4, 3)}
 -x_{Edge(4, 3)} + 2*x_{Edge(3, 1),Edge(4, 2)}
 -x_{Edge(4, 3)} + 2*x_{Edge(3, 1),Edge(4, 1),Edge(4, 3)}
 -x_{Edge(4, 3)} + 2*x_{Edge(3, 2),Edge(4, 1)}
 -x_{Edge(4, 3)} + 2*x_{Edge(3, 2),Edge(4, 2),Edge(4, 3)}

In [34]:
Mat3 = matrix(QQ,[[(-1)^k*vol_map(b1*b^(rank(M)-2k-1)*b2) for b1 in basis_HR] for b2 in basis_HR])

[6   2    4   2    4   2    4]
[2   6    4   2    4   2    4]
[4   4   10   4    6   4    6]
[2   2    4   6    4   2    4]
[4   4    6   4   10   4    6]
[2   2    4   2    4   6    4]
[4   4    6   4    6   4   10]

In [35]:
is_positive_definite(matrix(ZZ, [ZZ(i) for i in Mat3]))

true

In [36]:
reduced_characteristic_polynomial(M)

q^2 - 5*q + 6

In [37]:
[ vol_map(a^(rank(M)-j-1)*b^j) for j in range(0,rank(M)-1) ]

3-element Vector{QQFieldElem}:
 1
 5
 6

## Epilogue

Sometimes it is too tedious to maintain the full names of the elements of original ground set.  Here is a trick which maps those elements to $1,2,3,\ldots$, keeping the original ordering.

In [38]:
MM = Matroid(M.pm_matroid);
cyclic_flats(MM)

6-element Vector{Vector{Int64}}:
 []
 [1, 2, 3]
 [1, 4, 5]
 [2, 4, 6]
 [3, 5, 6]
 [1, 2, 3, 4, 5, 6]

In [39]:
chow_ring(MM)

Quotient
  of multivariate polynomial ring in 13 variables x_{1}, x_{2}, x_{3}, x_{4}, ..., x_{3,5,6}
    over rational field
  by ideal with 65 generators